# 1. Data preprocessing

The goal of this worksheet is to fully process the dataset so the data can be used for modeling.

This worksheet is structured as following:
1. Obtaining the data
2. Inspecting the data
3. Cleaning and filtering the data
4. Feature engineering
5. Creating the target variable
6. Data selection for modeling


In [54]:
# Imports and display settings.
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Obtaining the data

The dataset is quite large so it should be only downloaded once. However if you plan to work on it continuously then consider re-downloading it once per month as Bondora adds more loans and some of the old loans are updated. After downloading we extract the zip file and load the extracted .csv file into a DataFrame. The data is also sorted in chronological order.

In [55]:
def download():

    import zipfile
    from requests import get
    url = "https://bondora.com/marketing/media/LoanData.zip"

    with open("/data/Data.zip", "wb") as file:
        response = get(url)
        file.write(response.content)

    with zipfile.ZipFile("/data/Data.zip","r") as zip_ref:
        zip_ref.extractall("data/")

#download()

In [56]:
data = pd.read_csv("data/LoanData.csv")
data = data.sort_values(by=['ListedOnUTC'])

C:\Users\danie\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (34,38,67,80,82,83,89,90) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Inspecting the data

We can see that the dataset contains ~200000 loans and has 112 different columns. The earliest loans are from 2009, that means we have over 10 years of loan data.

In [57]:
print(f"DataFrame shape: {data.shape}")

DataFrame shape: (197625, 112)


In [58]:
data.head(5)

,ReportAsOfEOD,LoanId,LoanNumber,ListedOnUTC,BiddingStartedOn,BidsPortfolioManager,BidsApi,BidsManual,PartyId,NewCreditCustomer,LoanApplicationStartedDate,LoanDate,ContractEndDate,FirstPaymentDate,MaturityDate_Original,MaturityDate_Last,ApplicationSignedHour,ApplicationSignedWeekday,VerificationType,LanguageCode,Age,DateOfBirth,Gender,Country,AppliedAmount,Amount,Interest,LoanDuration,MonthlyPayment,County,City,UseOfLoan,Education,MaritalStatus,NrOfDependants,EmploymentStatus,EmploymentDurationCurrentEmployer,EmploymentPosition,WorkExperience,OccupationArea,HomeOwnershipType,IncomeFromPrincipalEmployer,IncomeFromPension,IncomeFromFamilyAllowance,IncomeFromSocialWelfare,IncomeFromLeavePay,IncomeFromChildSupport,IncomeOther,IncomeTotal,ExistingLiabilities,LiabilitiesTotal,RefinanceLiabilities,DebtToIncome,FreeCash,MonthlyPaymentDay,ActiveScheduleFirstPaymentReached,PlannedPrincipalTillDate,PlannedInterestTillDate,LastPaymentOn,CurrentDebtDaysPrimary,DebtOccuredOn,CurrentDebtDaysSecondary,DebtOccuredOnForSecondary,ExpectedLoss,LossGivenDefault,ExpectedReturn,ProbabilityOfDefault,DefaultDate,PrincipalOverdueBySchedule,PlannedPrincipalPostDefault,PlannedInterestPostDefault,EAD1,EAD2,PrincipalRecovery,InterestRecovery,RecoveryStage,StageActiveSince,ModelVersion,Rating,EL_V0,Rating_V0,EL_V1,Rating_V1,Rating_V2,Status,Restructured,ActiveLateCategory,WorseLateCategory,CreditScoreEsMicroL,CreditScoreEsEquifaxRisk,CreditScoreFiAsiakasTietoRiskGrade,CreditScoreEeMini,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PrincipalWriteOffs,InterestAndPenaltyWriteOffs,PrincipalBalance,InterestAndPenaltyBalance,NoOfPreviousLoansBeforeLoan,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsBefoleLoan,PreviousEarlyRepaymentsCountBeforeLoan,GracePeriodStart,GracePeriodEnd,NextPaymentDate,NextPaymentNr,NrOfScheduledPayments,ReScheduledOn,PrincipalDebtServicingCost,InterestAndPenaltyDebtServicingCost,ActiveLateLastPaymentCategory
957,2021-11-10,FA160D69-2682-4A60-8D8E-9BB700EA30CE,37,2009-02-21 14:12:39,2009-02-21 14:12:39,0,0.00,63.91,{544DFBAC-374F-4039-AE45-9BB700E44853},True,2009-02-21 14:12:39,2009-03-07,2009-09-10,2009-04-10,2009-09-10,2009-09-10,14,7,2.00,1,21,NaN,0.00,EE,63.91,63.91,20.00,6,NaN,NaN,NaN,3,5.00,4.00,0.00,5.00,UpTo2Years,NaN,5To10Years,NaN,NaN,15500.00,0.00,0.00,0.00,0.00,0.00,0.00,15500.00,0,0.00,0,0.00,0.00,10,True,1000.00,34.14,2009-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Repaid,False,NaN,NaN,NaN,NaN,NaN,NaN,63.91,2.18,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN
958,2021-11-10,8E929B92-7C99-421D-8499-9BB70125F390,42,2009-02-21 17:50:14,2009-02-21 17:50:14,0,0.00,83.09,{B7FDCB11-4CE9-4CDE-993F-9BB70103B180},True,2009-02-21 17:50:14,2009-03-03,2011-03-14,2009-04-13,2011-03-14,2011-03-14,17,7,2.00,1,19,NaN,1.00,EE,958.67,83.08,10.00,24,NaN,NaN,NaN,6,2.00,3.00,2.00,3.00,UpTo1Year,NaN,LessThan2Years,NaN,NaN,4000.00,0.00,0.00,0.00,0.00,0.00,0.00,4000.00,0,0.00,0,0.00,0.00,12,True,1300.00,142.73,2011-03-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Repaid,False,NaN,180+,NaN,NaN,NaN,NaN,83.08,9.20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN
959,2021-11-10,33B3F669-D0E3-4474-8045-9BB70128D064,43,2009-02-21 18:00:40,2009-02-21 18:00:40,0,0.00,322.75,{E58803E6-77B6-40EB-83C4-9BB70118C245},True,2009-02-21 18:00:40,2009-02-28,2010-03-10,2009-04-10,2010-03-10,2010-03-10,18,7,2.00,1,38,NaN,0.00,EE,639.12,322.75,25.00,12,NaN,NaN,NaN,6,4.00,4.00,4.00,3.00,UpTo4Years,NaN,15To25Years,NaN,NaN,9000.00,0.00,0.00,0.00,0.00,0.00,0.00,9000.00,0,0.00,0,0.00,0.00,10,True,5050.00,697.28,2010-03-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Repaid,False,NaN,151-180,NaN,NaN,NaN,NaN,322.75,44.56,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,0.00

In [59]:
data.tail(5)

,ReportAsOfEOD,LoanId,LoanNumber,ListedOnUTC,BiddingStartedOn,BidsPortfolioManager,BidsApi,BidsManual,PartyId,NewCreditCustomer,LoanApplicationStartedDate,LoanDate,ContractEndDate,FirstPaymentDate,MaturityDate_Original,MaturityDate_Last,ApplicationSignedHour,ApplicationSignedWeekday,VerificationType,LanguageCode,Age,DateOfBirth,Gender,Country,AppliedAmount,Amount,Interest,LoanDuration,MonthlyPayment,County,City,UseOfLoan,Education,MaritalStatus,NrOfDependants,EmploymentStatus,EmploymentDurationCurrentEmployer,EmploymentPosition,WorkExperience,OccupationArea,HomeOwnershipType,IncomeFromPrincipalEmployer,IncomeFromPension,IncomeFromFamilyAllowance,IncomeFromSocialWelfare,IncomeFromLeavePay,IncomeFromChildSupport,IncomeOther,IncomeTotal,ExistingLiabilities,LiabilitiesTotal,RefinanceLiabilities,DebtToIncome,FreeCash,MonthlyPaymentDay,ActiveScheduleFirstPaymentReached,PlannedPrincipalTillDate,PlannedInterestTillDate,LastPaymentOn,CurrentDebtDaysPrimary,DebtOccuredOn,CurrentDebtDaysSecondary,DebtOccuredOnForSecondary,ExpectedLoss,LossGivenDefault,ExpectedReturn,ProbabilityOfDefault,DefaultDate,PrincipalOverdueBySchedule,PlannedPrincipalPostDefault,PlannedInterestPostDefault,EAD1,EAD2,PrincipalRecovery,InterestRecovery,RecoveryStage,StageActiveSince,ModelVersion,Rating,EL_V0,Rating_V0,EL_V1,Rating_V1,Rating_V2,Status,Restructured,ActiveLateCategory,WorseLateCategory,CreditScoreEsMicroL,CreditScoreEsEquifaxRisk,CreditScoreFiAsiakasTietoRiskGrade,CreditScoreEeMini,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PrincipalWriteOffs,InterestAndPenaltyWriteOffs,PrincipalBalance,InterestAndPenaltyBalance,NoOfPreviousLoansBeforeLoan,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsBefoleLoan,PreviousEarlyRepaymentsCountBeforeLoan,GracePeriodStart,GracePeriodEnd,NextPaymentDate,NextPaymentNr,NrOfScheduledPayments,ReScheduledOn,PrincipalDebtServicingCost,InterestAndPenaltyDebtServicingCost,ActiveLateLastPaymentCategory
197112,2021-11-10,9443551E-9656-4C9C-B036-ADDB017E027D,2438363,2021-11-09 21:14:48,2021-11-09 23:14:48,28,3.00,16.00,{7B3BFFB4-9CA2-4AD8-A38F-AD3300342FCB},True,2021-11-09 23:10:51,2021-11-09,NaN,2021-12-08,2026-11-09,2026-11-09,23,3,4.00,1,31,NaN,0.00,EE,637.00,637.00,43.26,60,25.34,NaN,NaN,-1,1.00,-1.00,NaN,-1.00,UpTo1Year,NaN,NaN,-1.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1400.00,0,0.00,0,0.00,0.00,8,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.20,0.55,0.06,0.36,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,F,NaN,NaN,NaN,NaN,NaN,Current,False,NaN,NaN,M,NaN,NaN,900.00,0.00,0.00,NaN,NaN,637.00,NaN,0.00,0.00,NaN,NaN,0.00,NaN,NaN,2021-12-08,1.00,60.00,NaN,NaN,NaN,NaN
197111,2021-11-10,96A41405-759B-4936-BA22-ADDB017B8C18,2438353,2021-11-09 21:23:44,2021-11-09 23:23:44,127,0.00,91.00,{9E336188-56C7-4FDB-B370-ADDB017B8C46},True,2021-11-09 23:01:54,2021-11-09,NaN,2021-11-15,2026-10-13,2026-10-13,23,3,4.00,4,22,NaN,0.00,FI,3113.00,3113.00,18.98,60,77.82,NaN,NaN,-1,3.00,-1.00,NaN,-1.00,UpTo5Years,NaN,NaN,-1.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1600.00,0,0.00,0,0.00,0.00,13,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.09,0.73,0.07,0.12,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,D,NaN,NaN,NaN,NaN,NaN,Current,False,NaN,NaN,M,NaN,4,NaN,0.00,0.00,NaN,NaN,3113.00,NaN,0.00,0.00,NaN,NaN,0.00,NaN,NaN,2021-11-15,1.00,60.00,NaN,NaN,NaN,NaN
197113,2021-11-10,337C80A1-882F-4E27-A199-ADDB01861DD6,2438387,2021-11-09 21:46:45,2021-11-09 23:46:45,15,0.00,56.00,{16C016AA-FEE1-4BA1-8F8A-ACED00B658DF},True,2021-11-09 23:40:22,2021-11-09,NaN,2021-12-08,2024-11-08,2024-11-08,23,3,4.00,1,64,NaN,1.00,EE,956.00,956.00,34.05,36,43.59,NaN,NaN,-1,5.00,-1.00,NaN,-1.00,Retiree,NaN,NaN,-1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,656.00,3,248.27,0,0.00,0.00,8,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.11,0.60,0.13,0.18,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,D,NaN,NaN,NaN,NaN,NaN,Current,False,NaN,NaN,M,NaN,NaN,1000.00,0.00,0.00,NaN,NaN,956.00,NaN,1.00,2126.00,140.00,NaN,0.00,NaN,NaN,2021-12-08,1.00,36.00,NaN,NaN,NaN,NaN
197114,202

The data contains various numerical and categorical columns. To fully understand the data and proceed with cleaning the dataset, one must work through most of the columns. Visual inspection is important when working with this dataset as some columns have become obsolete halfway through the dataset. It is also helpful to statistically describe different columns as it helps to get a better understanding of the data. Below are some examples on how some of the numerical and categorical columns were analyzed.

Another helpful resource is the overview of the codes and terms that are used in the dataset which can be found at the [public-reports](https://www.bondora.com/en/public-reports) section on Bondora's website.

In [60]:
# Describing a numerical column
data.Amount.describe()

count   197625.00
mean      2572.08
std       2199.15
min          6.39
25%        740.00
50%       2125.00
75%       4100.00
max      10632.00
Name: Amount, dtype: float64

In [61]:
# Numerical column value counts
data.Amount.value_counts(dropna=False)

530.00     20337
531.00     13920
4150.00    11813
2125.00     9046
2126.00     7108
           ...  
6576.00        1
4623.00        1
5263.00        1
4174.00        1
1178.90        1
Name: Amount, Length: 6606, dtype: int64

In [62]:
# Describing a categorical column
data.Rating.describe()

count     194892
unique         8
top            D
freq       46991
Name: Rating, dtype: object

In [63]:
# Categorical column value counts
data.Rating.value_counts(dropna=False)

D      46991
E      34391
C      31424
F      26159
B      24633
HR     14832
A       8798
AA      7664
NaN     2733
Name: Rating, dtype: int64

## Cleaning and filtering the data

The first step is to drop all the unnecessary columns. Many of these columns are obsolete (see Bondora API docs) or serve no purpose for us. However some of the columns are not marked as obsolete in the API docs and half-way through the dataset they are filled only with null values. The reason behind this is most likely EU's data protection law (GDPR). These columns were detected by visually going through the dataset and since the last half of the data is missing in these columns, there is really no point in keeping them.

In [64]:
# Drop unnecessary columns.
df = data.drop(['BiddingStartedOn', 'LoanApplicationStartedDate', 'ApplicationSignedHour',
                'ApplicationSignedWeekday', 'DateOfBirth', 'County', 'City', 'UseOfLoan',
                'MaritalStatus', 'NrOfDependants', 'EmploymentStatus', 'EmploymentPosition',
                'WorkExperience', 'OccupationArea', 'IncomeFromPrincipalEmployer', 'IncomeFromPension',
                'IncomeFromFamilyAllowance', 'IncomeFromSocialWelfare', 'IncomeFromLeavePay',
                'IncomeFromChildSupport', 'IncomeOther', 'RefinanceLiabilities', 'DebtToIncome', 'FreeCash', 'MonthlyPaymentDay',
                'EL_V0', 'Rating_V0', 'EL_V1', 'Rating_V1', 'Rating_V2', 'PrincipalWriteOffs', 'InterestAndPenaltyWriteOffs', 'PlannedPrincipalTillDate', 'CreditScoreEsEquifaxRisk', 'CreditScoreEsMicroL', 'BidsPortfolioManager', 'BidsApi', 'BidsManual', 'PrincipalDebtServicingCost', 'InterestAndPenaltyDebtServicingCost',
                'ContractEndDate', 'LoanNumber', 'FirstPaymentDate', 'PlannedInterestTillDate', 'LastPaymentOn', 'CurrentDebtDaysPrimary',
                'DebtOccuredOn', 'CurrentDebtDaysSecondary', 'DebtOccuredOnForSecondary', 'ExpectedLoss', 'LossGivenDefault', 'ExpectedReturn',
                'ProbabilityOfDefault', 'ActiveScheduleFirstPaymentReached', 'PlannedPrincipalPostDefault', 'PlannedInterestPostDefault', 'EAD1', 'EAD2',
                'PrincipalRecovery', 'InterestRecovery', 'RecoveryStage', 'StageActiveSince', 'ModelVersion', 'NextPaymentNr', 'ReScheduledOn'], axis=1)

# Set "LoanId" as the index
df = df.set_index('LoanId')

In order to further reduce the size of the dataset we apply some filters. We parse some of the columns into date format, so it makes it easier to use them in the filters.

Since we do not know the outcomes of active loans we do not include them. However if the loan is active but has been restructured, then we keep it and use it in the target variable as a "bad" loan since we know for sure that the loan has had issues. The reason for not including loans that are active and have not had any issues is that we have no way of knowing if issues would or would not arise in the future.

We also apply some filters to numerical and categorical columns in order to deal with anomalies. And we also deal with null values.

In [65]:
data.ActiveScheduleFirstPaymentReached.value_counts(dropna=False)

True     184208
False     13417
Name: ActiveScheduleFirstPaymentReached, dtype: int64

In [66]:
df.head(5)

,ReportAsOfEOD,ListedOnUTC,PartyId,NewCreditCustomer,LoanDate,MaturityDate_Original,MaturityDate_Last,VerificationType,LanguageCode,Age,Gender,Country,AppliedAmount,Amount,Interest,LoanDuration,MonthlyPayment,Education,EmploymentDurationCurrentEmployer,HomeOwnershipType,IncomeTotal,ExistingLiabilities,LiabilitiesTotal,DefaultDate,PrincipalOverdueBySchedule,Rating,Status,Restructured,ActiveLateCategory,WorseLateCategory,CreditScoreFiAsiakasTietoRiskGrade,CreditScoreEeMini,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PrincipalBalance,InterestAndPenaltyBalance,NoOfPreviousLoansBeforeLoan,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsBefoleLoan,PreviousEarlyRepaymentsCountBeforeLoan,GracePeriodStart,GracePeriodEnd,NextPaymentDate,NrOfScheduledPayments,ActiveLateLastPaymentCategory
LoanId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
FA160D69-2682-4A60-8D8E-9BB700EA30CE,2021-11-10,2009-02-21 14:12:39,{544DFBAC-374F-4039-AE45-9BB700E44853},True,2009-03-07,2009-09-10,2009-09-10,2.00,1,21,0.00,EE,63.91,63.91,20.00,6,NaN,5.00,UpTo2Years,NaN,15500.00,0,0.00,NaN,NaN,NaN,Repaid,False,NaN,NaN,NaN,NaN,63.91,2.18,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN
8E929B92-7C99-421D-8499-9BB70125F390,2021-11-10,2009-02-21 17:50:14,{B7FDCB11-4CE9-4CDE-993F-9BB70103B180},True,2009-03-03,2011-03-14,2011-03-14,2.00,1,19,1.00,EE,958.67,83.08,10.00,24,NaN,2.00,UpTo1Year,NaN,4000.00,0,0.00,NaN,NaN,NaN,Repaid,False,NaN,180+,NaN,NaN,83.08,9.20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN
33B3F669-D0E3-4474-8045-9BB70128D064,2021-11-10,2009-02-21 18:00:40,{E58803E6-77B6-40EB-83C4-9BB70118C245},True,2009-02-28,2010-03-10,2010-03-10,2.00,1,38,0.00,EE,639.12,322.75,25.00,12,NaN,4.00,UpTo4Years,NaN,9000.00,0,0.00,NaN,NaN,NaN,Repaid,False,NaN,151-180,NaN,NaN,322.75,44.56,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN
7074D9E8-E8F5-403B-8614-9BB701338AD4,2021-11-10,2009-02-21 18:39:43,{DE67C0EB-7534-47F2-87BD-9BB7011E112A},True,2009-03-03,2011-04-01,2011-04-01,2.00,1,55,1.00,EE,958.67,252.45,10.00,24,NaN,3.00,MoreThan5Years,NaN,6000.00,0,0.00,NaN,NaN,NaN,Repaid,False,NaN,121-150,NaN,NaN,252.45,29.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN
39F2A312-CD6C-4F60-A7F6-9BB7014CE0A9,2021-11-10,2009-02-21 20:11:58,{C41C37A5-B2D7-4B5A-B2D2-9BB70149B419},True,2009-03-07,2010-04-06,2010-04-06,2.00,1,46,1.00,EE,766.94,63.91,20.00,12,NaN,5.00,MoreThan5Years,NaN,10700.00,0,0.00,2010-05-04,NaN,NaN,Repaid,False,NaN,61-90,NaN,NaN,63.91,8.65,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN


In [67]:
# Parse date columns into date format.
date_columns = ['LoanDate', 'MaturityDate_Original', 'MaturityDate_Last', 'ReportAsOfEOD']
df[date_columns] = df[date_columns].apply(pd.to_datetime, format='%Y-%m-%d', errors='coerce')

# Do not include loans that are currently active, and have not been restructured (the original maturity date of the loan has not increased by at least 60 days).
df = df.loc[~((df.Status == 'Current') & (df.Restructured == 0))]

# Some loans have been repaid just days after signing a contract so they should be filtered out.
df = df.loc[~(((df.MaturityDate_Last - df.LoanDate) < pd.to_timedelta("180days")) & (df.Status == 'Repaid'))]

# Remove the most recent loans because most of them seem to be with the status "Late".
df = df.loc[(df.LoanDate < (df.ReportAsOfEOD - pd.to_timedelta("60days")))]

In [68]:
# Filters regarding numerical values.

# Age must be between 18 and 70.
df = df.loc[(df.Age >= 18) & (df.Age <= 70)]

# Monthly payment must be over 0.
df = df.loc[(df.MonthlyPayment > 0)]

In [69]:
# Filters regarding categorical values.

# Select allowed loan durations. This dataset contains various contract lengths, but we are interested in the most common ones. Values represent loan duration in months.
df.LoanDuration = df.LoanDuration.astype('category')
df = df.loc[(df.LoanDuration.isin([60, 48, 36, 30, 24, 18, 12, 9, 6]))]

# We only want loans from EE.
df = df.loc[(df.Country.isin(['EE']))]

# The main verification types.
df.VerificationType = df.VerificationType.astype('category')
df = df.loc[(df.VerificationType.isin([1, 2, 3, 4]))]

# Gender is either male of female.
df.Gender = df.Gender.astype('category')
df = df.loc[(df.Gender.isin([0, 1]))]

# Select allowed education levels.
df.Education = df.Education.astype('category')
df = df.loc[(df.Education.isin([1, 2, 3, 4, 5]))]

# Select allowed employment durations.
df = df.loc[(df.EmploymentDurationCurrentEmployer.isin(['MoreThan5Years', 'UpTo5Years', 'UpTo4Years', 'UpTo3Years', 'UpTo2Years', 'UpTo1Year', 'TrialPeriod', 'Retiree', 'Other']))]

# Select allowed home ownership types.
df.HomeOwnershipType = df.HomeOwnershipType.astype('category')
df = df.loc[(df.HomeOwnershipType.isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))]

# Select allowed ratings.
df = df.loc[(df.Rating.isin(['AA', 'A', 'B', 'C', 'D', 'E', 'F', 'HR']))]

# Select allowed credit score levels.
df.CreditScoreEeMini = df.CreditScoreEeMini.astype('category')
df = df.loc[(df.CreditScoreEeMini.isin([1000, 900, 800, 700, 600, 500]))]

In [70]:
# Dealing with null values

df.PreviousRepaymentsBeforeLoan = df.PreviousRepaymentsBeforeLoan.fillna(0)

df.PreviousEarlyRepaymentsBefoleLoan = df.PreviousEarlyRepaymentsBefoleLoan.fillna(0)

In [71]:
print(f"DataFrame shape after cleaning: {df.shape}")

DataFrame shape after cleaning: (82291, 46)


We can see that the dataset is contains alot less rows and columns after processing it.

## Feature engineering

TODO

## Constructing the target variable

We construct a target variable called "PreferLoan" which is a boolean value. As stated before, loans with previous issues are not to be preferred and are assigned the value "0". Loans without issues are assigned the value "1". However there is a small exception. If the the worst late category is 16-30 days or less and the loan has not had any other issues then the loan is also preferred.

In [72]:
# Constructing the target value.

# Loan status must be 'Repaid'.
# WorseLateCategory must not be higher than 16-30 (can be null).
# Loan must be repaid before or on the original maturity date.
# Loan must not be restructured.
# Loan must not be defaulted.

# Set the default value for all loans to be 0.
df["PreferLoan"] = 0

# Select preferred loans and set their value to 1.
df.loc[(
               (df.Status == 'Repaid') &
               (df.WorseLateCategory.isin([np.nan, '1-7', '8-15', '16-30'])) &
               (df.MaturityDate_Last <= df.MaturityDate_Original) &
               (df.Restructured != 1) &
               (df.DefaultDate.isnull())
       ), 'PreferLoan'] = 1

## Next steps
We have successfully cleaned and processed the dataset.It can be used in the next steps which are modelling and validating the model.

In [76]:
df.to_csv('data/processed.csv', index=True)